In [29]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [30]:
df = pd.read_csv('/content/drive/MyDrive/기상청/최종작업/추가 날씨 데이터_결측치제거전_2020_2023.csv')
df.head()

,tm,address_city,address_gu,sub_address,stn,ta_max,ta_min,ta_max_min,hm_min,hm_max,...,지역코드,생활인구_전체,생활인구_남,생활인구_여,노년층_생활인구,세대수,사고,사망,부상,공휴일
0,20200501,부산광역시,강서구,대저2동,904,23.7,16.6,7.1,63.2,90.3,...,2644000000,185926,104856,81070,30948,54994,1,0,1,0
1,20200501,부산광역시,강서구,천성동,921,24.3,15.4,8.9,64.2,96.4,...,2644000000,185926,104856,81070,30948,54994,1,0,1,0
2,20200501,부산광역시,금정구,금사동,940,25.6,17.2,8.4,46.6,76.4,...,2641000000,278185,134269,143916,64689,107313,2,0,2,0
3,20200501,부산광역시,금정구,금성동,941,25.3,17.1,8.2,46.6,76.4,...,2641000000,278185,134269,143916,64689,107313,2,0,2,0
4,20200501,부산광역시,금정구,남산동,939,26.5,16.7,9.8,46.6,76.4,...,2641000000,278185,134269,143916,64689,107313,2,0,2,0


# 결측치처리

In [31]:
df.isnull().sum()

,0
tm,0
address_city,0
address_gu,0
sub_address,0
stn,0
ta_max,0
ta_min,0
ta_max_min,0
hm_min,0
hm_max,0


In [32]:
# 병합 전 컬럼 순서 저장
original_columns = df.columns.tolist()

## rn_day 결측치 처리
- 같은 구의 평균
- 없으면, 같은 날짜의 전체 평균

In [33]:
# 1차: 같은 날짜, 같은 구의 평균으로 채우기
df['rn_day'] = df['rn_day'].fillna(
    df.groupby(['tm', 'address_gu'])['rn_day'].transform('mean')
)

# 2차: 같은 날짜의 전체 평균으로 채우기 (address_gu 없이)
df['rn_day'] = df['rn_day'].fillna(
    df.groupby('tm')['rn_day'].transform('mean')
)

df[['rn_day']].isnull().sum()

,0
rn_day,0


## 평균 풍속, 풍정합 결측치 처리
- 앞, 뒤 날짜 선형 보간

In [34]:
# 필요한 열만 추출 (평균 풍속, 풍정합, 날짜)
df_wind = df[['tm', '평균 풍속(m/s)', '풍정합(100m)']].drop_duplicates()

# 날짜 정렬 및 선형 보간
df_wind = df_wind.sort_values('tm')
df_wind[['평균 풍속(m/s)', '풍정합(100m)']] = df_wind[['평균 풍속(m/s)', '풍정합(100m)']].interpolate(method='linear', limit_direction='both')

# 보간된 값 원본 df에 병합 (tm 기준으로 join)
df = df.drop(['평균 풍속(m/s)', '풍정합(100m)'], axis=1).merge(df_wind, on='tm', how='left')

df.head()

,tm,address_city,address_gu,sub_address,stn,ta_max,ta_min,ta_max_min,hm_min,hm_max,...,생활인구_남,생활인구_여,노년층_생활인구,세대수,사고,사망,부상,공휴일,평균 풍속(m/s),풍정합(100m)
0,20200501,부산광역시,강서구,대저2동,904,23.7,16.6,7.1,63.2,90.3,...,104856,81070,30948,54994,1,0,1,0,5.8,5001.0
1,20200501,부산광역시,강서구,천성동,921,24.3,15.4,8.9,64.2,96.4,...,104856,81070,30948,54994,1,0,1,0,5.8,5001.0
2,20200501,부산광역시,금정구,금사동,940,25.6,17.2,8.4,46.6,76.4,...,134269,143916,64689,107313,2,0,2,0,5.8,5001.0
3,20200501,부산광역시,금정구,금성동,941,25.3,17.1,8.2,46.6,76.4,...,134269,143916,64689,107313,2,0,2,0,5.8,5001.0
4,20200501,부산광역시,금정구,남산동,939,26.5,16.7,9.8,46.6,76.4,...,134269,143916,64689,107313,2,0,2,0,5.8,5001.0


## 평균 이슬점온도, 평균 상대습도, 평균 증기압 결측치 처리
- 2023년 5월 29, 30일 이틀치 자료 부족
- 평균 이슬점온도, 평균 상대습도 -> 울산 공항의 이슬점 온도와 상대습도
- 평균 온도는 해당일자 부산 광역시 평균 온도 데이터 가져와서 평균 증기압 계산

In [35]:
# === 상수 ===
a = 17.62
b = 243.12

# === 날짜별 입력값 ===
data_dict = {
    20230529: {
        'temp_avg': 20.2,
        'dew_temp': 20.0,
        'rh': 98
    },
    20230530: {
        'temp_avg': 18.6,
        'dew_temp': 16.9,
        'rh': 88
    }
}

# === 계산 및 적용 ===
for date, values in data_dict.items():
    temp_avg = values['temp_avg']
    dew_temp = values['dew_temp']
    rh = values['rh']

    # 평균 상대습도 입력
    df.loc[df['tm'] == date, '평균 상대습도(%)'] = rh

    # 평균 이슬점온도 입력
    df.loc[df['tm'] == date, '평균 이슬점온도(°C)'] = dew_temp

    # 평균 증기압 계산
    vapor_pressure = 6.112 * np.exp((a * dew_temp) / (b + dew_temp))
    df.loc[df['tm'] == date, '평균 증기압(hPa)'] = vapor_pressure


## 최저 해면기압 결측치 처리
- 최고 해면기압과, 평균 해면기압이 존재 -> 결측치 없음
  - 평균 해면 기압 * 2 - 최고 해면 기압으로 최저 해면기압 계산
- 데이터 흐름을 봤을 때, 선형 보간 방법보다 더 오차가 적다고 판단

In [36]:
# 결측치 마스크
mask = df['최저 해면기압(hPa)'].isnull()

# 평균과 최고 해면기압을 이용해 추정값 계산 및 대입
df.loc[mask, '최저 해면기압(hPa)'] = (
    2 * df.loc[mask, '평균 해면기압(hPa)'] - df.loc[mask, '최고 해면기압(hPa)']
)

## 합계 일조시간, 1시간 최다일사량, 합계 일사량 결측치 처리
- 이틀의 결측치가 존재
  - 첫 날의 경우 전일과 해당일 비, 다음일 맑음
  - 두 번째 날의 경우 전일, 해당일, 다음일 모두 맑음 이후 갈수록 흐려지는 상황
- 두 날 모두 전 일의 기상 상황과 맞다고 판단해서 전 일의 수치로 결측치 처리


In [37]:
# 보정 대상 열
sun_cols = ['합계 일조시간(hr)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)']

# tm이 정수이면 datetime으로 변환해서 처리하기 쉽게
df['tm_dt'] = pd.to_datetime(df['tm'].astype(str), format='%Y%m%d')

# 날짜 기준으로 정렬 (안전하게)
df = df.sort_values('tm_dt')

# 결측치 있는 날짜 리스트
missing_days = df[df[sun_cols].isnull().any(axis=1)]['tm_dt'].unique()

# 자동 전날 복사
for target_day in missing_days:
    prev_day = target_day - timedelta(days=1)

    for col in sun_cols:
        # 전날 값이 존재하면 가져오기
        if prev_day in df['tm_dt'].values:
            source_val = df.loc[df['tm_dt'] == prev_day, col].iloc[0]
            df.loc[df['tm_dt'] == target_day, col] = source_val

## 평균 지면온도 결측치 처리
- 하루의 결측치 존재
  - 전 후로 비슷한 날씨로 존재하기 때문에, 선형 보간

In [38]:
# 날짜별 대표값만 추출
df_surface = df[['tm', '평균 지면온도(°C)']].drop_duplicates().sort_values('tm')

# 선형 보간 적용
df_surface['평균 지면온도(°C)'] = df_surface['평균 지면온도(°C)'].interpolate(method='linear', limit_direction='both')

# 원본에서 해당 컬럼 제거 후 병합
df = df.drop(columns=['평균 지면온도(°C)']).merge(df_surface, on='tm', how='left')

## 대형 증발량, 소형 증발량 결측치 처리
- 전, 후일 기준으로 선형 보간

In [39]:
# 1. 기존 컬럼 순서 저장
original_columns = df.columns.tolist()

# 2. 날짜별 증발량만 추출
df_evap = df[['tm', '합계 대형증발량(mm)', '합계 소형증발량(mm)']].drop_duplicates().sort_values('tm')

# 3. 선형 보간 적용
df_evap[['합계 대형증발량(mm)', '합계 소형증발량(mm)']] = df_evap[[
    '합계 대형증발량(mm)', '합계 소형증발량(mm)'
]].interpolate(method='linear', limit_direction='both')

# 4. 병합
df = df.drop(columns=['합계 대형증발량(mm)', '합계 소형증발량(mm)']).merge(df_evap, on='tm', how='left')

## 안계 계속시간 결측치 처리
- 비어있는 날은 0으로

In [40]:
df['안개 계속시간(hr)'].fillna(0, inplace=True)

<ipython-input-40-1979166580>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['안개 계속시간(hr)'].fillna(0, inplace=True)


## 컬럼 기존 순서 복원

In [41]:
# 1. 필요 시 임시 컬럼 제거
if 'tm_dt' in df.columns:
    df = df.drop(columns=['tm_dt'])

# 2. 실제 존재하는 컬럼만 필터링하여 순서 복원
df = df[[col for col in original_columns if col in df.columns]]

In [42]:
df

,tm,address_city,address_gu,sub_address,stn,ta_max,ta_min,ta_max_min,hm_min,hm_max,...,생활인구_여,노년층_생활인구,세대수,사고,사망,부상,공휴일,평균 풍속(m/s),풍정합(100m),평균 지면온도(°C)
0,20200501,부산광역시,강서구,대저2동,904,23.7,16.6,7.1,63.2,90.3,...,81070,30948,54994,1,0,1,0,5.8,5001.0,26.1
1,20200501,부산광역시,사상구,엄궁동,904,23.7,16.6,7.1,63.2,90.3,...,119597,51369,97038,3,0,4,0,5.8,5001.0,26.1
2,20200501,부산광역시,사상구,학장동,904,23.7,16.6,7.1,63.2,90.3,...,119597,51369,97038,3,0,4,0,5.8,5001.0,26.1
3,20200501,부산광역시,사하구,감천동,159,22.7,16.7,6.0,63.7,87.5,...,161633,69622,138962,2,0,3,0,5.8,5001.0,26.1
4,20200501,부산광역시,사하구,다대동,950,23.6,15.4,8.2,61.9,93.8,...,161633,69622,138962,2,0,3,0,5.8,5001.0,26.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42919,20231031,부산광역시,동래구,온천동,940,24.5,10.0,14.5,20.4,94.0,...,137369,72161,119841,2,0,2,0,3.1,2720.0,21.4
42920,20231031,부산광역시,부산진구,당감동,938,23.7,10.4,13.3,20.5,91.9,...,202689,104283,181565,0,0,0,0,3.1,2720.0,21.4
42921,20231031,부산광역시,부산진구,부암동,938,23.7,10.4,13.3,20.5,91.9,...,202689,104283,181565,0,0,0,0,3.1,2720.0,21.4
42922,20231031,부산광역시,남구,감만동,942,24.0,12.0,12.0,23.0,88.0,...,134909,73044,118390,2,0,2,0,3.1,2720.0,21.4


In [43]:
# 지역코드 제거
df = df.drop('지역코드', axis=1)

In [44]:
df.columns

Index(['tm', 'address_city', 'address_gu', 'sub_address', 'stn', 'ta_max',
       'ta_min', 'ta_max_min', 'hm_min', 'hm_max', 'ws_max', 'ws_ins_max',
       'rn_day', 'call_count', '평균 이슬점온도(°C)', '평균 상대습도(%)', '평균 증기압(hPa)',
       '평균 현지기압(hPa)', '최고 해면기압(hPa)', '최저 해면기압(hPa)', '평균 해면기압(hPa)',
       '가조시간(hr)', '합계 일조시간(hr)', '1시간 최다일사량(MJ/m2)', '합계 일사량(MJ/m2)',
       '평균 전운량(1/10)', '평균 중하층운량(1/10)', '합계 대형증발량(mm)', '합계 소형증발량(mm)',
       '안개 계속시간(hr)', '시정(10m)', '생활인구_전체', '생활인구_남', '생활인구_여', '노년층_생활인구',
       '세대수', '사고', '사망', '부상', '공휴일', '평균 풍속(m/s)', '풍정합(100m)',
       '평균 지면온도(°C)'],
      dtype='object')

In [45]:
df.isnull().sum()

,0
tm,0
address_city,0
address_gu,0
sub_address,0
stn,0
ta_max,0
ta_min,0
ta_max_min,0
hm_min,0
hm_max,0


In [46]:
# 최종 데이터 저장 경로
file_path = "2020_2023_최종데이터.csv"

# CSV 파일로 저장
df.to_csv(file_path, index=False, encoding='utf-8')  # Excel 호환을 위해 utf-8-sig 권장

In [47]:
df

,tm,address_city,address_gu,sub_address,stn,ta_max,ta_min,ta_max_min,hm_min,hm_max,...,생활인구_여,노년층_생활인구,세대수,사고,사망,부상,공휴일,평균 풍속(m/s),풍정합(100m),평균 지면온도(°C)
0,20200501,부산광역시,강서구,대저2동,904,23.7,16.6,7.1,63.2,90.3,...,81070,30948,54994,1,0,1,0,5.8,5001.0,26.1
1,20200501,부산광역시,사상구,엄궁동,904,23.7,16.6,7.1,63.2,90.3,...,119597,51369,97038,3,0,4,0,5.8,5001.0,26.1
2,20200501,부산광역시,사상구,학장동,904,23.7,16.6,7.1,63.2,90.3,...,119597,51369,97038,3,0,4,0,5.8,5001.0,26.1
3,20200501,부산광역시,사하구,감천동,159,22.7,16.7,6.0,63.7,87.5,...,161633,69622,138962,2,0,3,0,5.8,5001.0,26.1
4,20200501,부산광역시,사하구,다대동,950,23.6,15.4,8.2,61.9,93.8,...,161633,69622,138962,2,0,3,0,5.8,5001.0,26.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42919,20231031,부산광역시,동래구,온천동,940,24.5,10.0,14.5,20.4,94.0,...,137369,72161,119841,2,0,2,0,3.1,2720.0,21.4
42920,20231031,부산광역시,부산진구,당감동,938,23.7,10.4,13.3,20.5,91.9,...,202689,104283,181565,0,0,0,0,3.1,2720.0,21.4
42921,20231031,부산광역시,부산진구,부암동,938,23.7,10.4,13.3,20.5,91.9,...,202689,104283,181565,0,0,0,0,3.1,2720.0,21.4
42922,20231031,부산광역시,남구,감만동,942,24.0,12.0,12.0,23.0,88.0,...,134909,73044,118390,2,0,2,0,3.1,2720.0,21.4


In [48]:
from sklearn.metrics import mean_squared_error
import numpy as np

# 실제 값
y_true = df['call_count']

# 예측값 (전부 1)
y_pred = np.ones_like(y_true)

# RMSE 계산
rmse = mean_squared_error(y_true, y_pred)
print(f"📉 RMSE (모두 1로 예측 시): {rmse:.4f}")

📉 RMSE (모두 1로 예측 시): 5.0792
